In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pd.read_csv("magic04.data")

In [ ]:
cols = ["fLength","fWidth","fSize","fConc","fConc1","fAsym","fM3Long","fM3Trans","fAlpha","fDist","class"]
df = pd.read_csv("magic04.data",names=cols)
df.head()

 here from "flength" to "class" called feature, feature are just thing that we are going to pass our model in order to help us predict the label,which in this case class column.

 here sample 0 has 10 different feature,so i have 10 different values that i can pass into some model.

In [ ]:
df["class"].unique()      #g-gamma h-hydra